In [13]:
import xarray as xr
import pandas as pd
import glob
import cftime

# List of files
file_list = sorted(glob.glob("C:/Users/jenna/OneDrive/Desktop/Fall 2025/Energy Meterology/Final Proj/data/CMIP SSP370 Near Surface/yearlydata/*.nc"))  # adjust path

# Target coordinates (BWI)
target_coords = {'lat': 39.17329, 'lon': -76.68404}
#city
#target_coords = {'lat': 39.2814, 'lon': -76.6111}

all_years = []

for f in file_list:
    ds = xr.open_dataset(f)
    
    # Fix longitude
    ds = ds.assign_coords(lon = (((ds.lon + 180) % 360) - 180)).sortby("lon")
    
    # Select nearest grid point
    point_data = ds['tas'].sel(lat=target_coords['lat'], lon=target_coords['lon'], method='nearest')
    
    # Convert to dataframe
    df_point = point_data.to_dataframe().reset_index()
    
    # Convert cftime to pandas datetime
    df_point['time'] = df_point['time'].apply(lambda t: pd.Timestamp(t.isoformat()))
    
    # Extract year if needed
    df_point['year'] = df_point['time'].dt.year
    
    all_years.append(df_point)

# Combine all years
combined_df = pd.concat(all_years, ignore_index=True)

print(combined_df)

           time     lat     lon         tas  year
0    2041-01-01  39.125 -76.625  288.319000  2041
1    2041-01-02  39.125 -76.625  283.161285  2041
2    2041-01-03  39.125 -76.625  286.867279  2041
3    2041-01-04  39.125 -76.625  286.955322  2041
4    2041-01-05  39.125 -76.625  281.806793  2041
...         ...     ...     ...         ...   ...
7295 2060-12-27  39.125 -76.625  281.596771  2060
7296 2060-12-28  39.125 -76.625  277.699249  2060
7297 2060-12-29  39.125 -76.625  274.136108  2060
7298 2060-12-30  39.125 -76.625  270.556366  2060
7299 2060-12-31  39.125 -76.625  277.773682  2060

[7300 rows x 5 columns]


In [14]:
# Convert tas from K to °C
combined_df['tas_C'] = combined_df['tas'] - 273.15
print(combined_df)


           time     lat     lon         tas  year      tas_C
0    2041-01-01  39.125 -76.625  288.319000  2041  15.169006
1    2041-01-02  39.125 -76.625  283.161285  2041  10.011292
2    2041-01-03  39.125 -76.625  286.867279  2041  13.717285
3    2041-01-04  39.125 -76.625  286.955322  2041  13.805328
4    2041-01-05  39.125 -76.625  281.806793  2041   8.656799
...         ...     ...     ...         ...   ...        ...
7295 2060-12-27  39.125 -76.625  281.596771  2060   8.446777
7296 2060-12-28  39.125 -76.625  277.699249  2060   4.549255
7297 2060-12-29  39.125 -76.625  274.136108  2060   0.986115
7298 2060-12-30  39.125 -76.625  270.556366  2060  -2.593628
7299 2060-12-31  39.125 -76.625  277.773682  2060   4.623688

[7300 rows x 6 columns]


In [15]:
import xarray as xr

# Convert dataframe to xarray Dataset
ds_out = combined_df.set_index('time').to_xarray()

# Save as NetCDF
ds_out.to_netcdf("SSP370_NearSurface_BWI.nc")